Things to cluster on:
Acceleration
angular velocity
accelartion towards target
angular velocity towards target



Distance
relative position and its deriv
velocity toward prey/pred == the velocity projection onto x-x and y-y, but without unit normalization, just dot product
relative angle
relative velocity
rate of change of relative heading angle.
self velocity.
self accelartion.

Cluster these things based on given time-windows.  How to choose dt, will save for later.
#discern going and resting by velocity thresholds....

# Methods:
Affinity propagation split between trial types: single target, multi-prey, prey-pred.
#need to get all sessions


In [1]:
import PacTimeOrig.DataHandling as DH
import PacTimeOrig.Methods.utils as pacutils
from PacTimeOrig.Methods.Clusteresque import recusiveCDF, EMDRecursive,getvarforclustering,loadEMDmatrices,EMDfast

import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from sklearn.cluster import AffinityPropagation

#set some analysis variables that'll stay fixed
winlength=10
n_sessions=5
for i in tqdm(np.arange(3,6,1)):
    session=i

    data=DH.dataloader(direc='C:\\Users\\Justin\\Documents\\GitHub\\PacManHuman\\',subject='H',session = session, suffix='Pac_dACC.mat')
    sessionVars = DH.ExperimentVarsRetrieve(data)
    positions = DH.retrievepositions(data)
    #velocity, accelaration
    positions = pacutils.computederivatives(positions)
    #distance and angle
    positions = pacutils.computeheading(positions)
    pacutils.computevelocitytowardstarget(positions)

    #Append the above info into SessionVars as a goodbad (good =1), get rection time, and idx of first peak for splitting
    sessionVars=pacutils.trialRtPkRej(sessionVars, positions)


    #Get aggregated features for each trial
    p1trialdat=pacutils.getclusterfeatures(sessionVars, positions,winlength=winlength)

    #Make sure not to include bad trials using the gb flag in sessionVars, let's make a new variable for it
    p1trials = sessionVars[sessionVars['numNPC'] == 1].index
    goodbad=sessionVars['goodtrial'].loc[p1trials].reset_index(drop=True)


    alldat=dict()
    alldat['positions']=positions
    alldat['sessionVars']=sessionVars
    alldat['p1trialdat']=p1trialdat
    alldat['p1trials']=p1trials
    alldat['goodbad']=goodbad
    #Save the matrices for later analysis
    import pickle

    with open('C:\\Users\\Justin\\Documents\\GitHub\\PacManHuman\\Results\\datum'+str(session)+'.pkl', 'wb') as handle:
        pickle.dump(alldat, handle, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/3 [00:00<?, ?it/s]

607


C:\Users\Justin\Documents\GitHub\PacManHuman\PacTimeOrig\Methods\utils.py:269: RuntimeWarning: invalid value encountered in true_divide
  filt_speed = filt_speed / np.max(filt_speed)
 33%|███▎      | 1/3 [00:28<00:57, 28.94s/it]

608


 67%|██████▋   | 2/3 [00:58<00:29, 29.08s/it]

509


C:\Users\Justin\Documents\GitHub\PacManHuman\PacTimeOrig\Methods\utils.py:269: RuntimeWarning: invalid value encountered in true_divide
  filt_speed = filt_speed / np.max(filt_speed)
100%|██████████| 3/3 [01:24<00:00, 28.19s/it]


#include:
1. subject speed
2. subject sum(acceleration.^2)
3. velocity
4. acceleration
5. velocity towards target -- as scalar projection
6. rate of change of scalar projection
6. relative angular heading
7. rate of change of angular heading
8. work or dot product of vector projection of velocity acceleration as the scalar projection onto the acceleration vector.
9. relative target quantities

In [ ]:
#Get speed distributions by target type (for normalization)
unval=np.unique(sessionVars.loc[p1trials]['NPCvalA']) #Get unique values
tmpval=sessionVars.loc[p1trials]['NPCvalA'] #Make PD df with just those values
tmpval.reset_index(drop=True,inplace=True) #and reset its index for using np.where

spdout=[]
for i in unval:
    tmptrial=np.where(tmpval==i)[0]
    tmpspd=pd.DataFrame()
    for trial in tmptrial:

        tmpspd=pd.concat((tmpspd,pd.DataFrame(p1trialdat[trial].prey1_scalar_projection)),axis=0)

    spdout.append(tmpspd)

#make a kernel density plot
sns.kdeplot(pd.DataFrame(spdout[0].to_numpy())[0])
sns.kdeplot(pd.DataFrame(spdout[1].to_numpy())[0])
sns.kdeplot(pd.DataFrame(spdout[2].to_numpy())[0])
sns.kdeplot(pd.DataFrame(spdout[3].to_numpy())[0])
sns.kdeplot(pd.DataFrame(spdout[4].to_numpy())[0])
plt.ylabel('density')
plt.xlabel('speed')
plt.legend()
plt.show()

In [ ]:
# 3/17/23

# Load each pikcled datum and concatante the P1 trials
datall=dict()
for i in range(5):
    session=i+1
    file = open('C:\\Users\\Justin\\Documents\\GitHub\\PacManHuman\\Results\\datum'+str(session)+'.pkl', 'rb')
    emdmatrices = pickle.load(file)
    datall[str(session)]=emdmatrices

# Then compute recursive Earth movers across all and store matrices for clustering.
#Fewer bins
histrange=dict()

histrange['speed']=np.arange(0,1400+300,400) # 4bins
histrange['speedgrad']=np.arange(-200,200+50,100) # 4bins
histrange['accelmag']=np.arange(0,20000+5000,5000) # 4bins
histrange['prey1reldistance']=np.arange(0,2000+400,400) # 5bins
histrange['prey1reldistancegrad']=np.arange(-40,40+10,20) #4 bins
histrange['prey1relangularhead']= np.arange(-180,180+45,45)
histrange['prey1relangularheadgrad']= np.arange(-30,30+30,15)
histrange['prey1_scalar_projection']=np.arange(-1500,1500+500,500)
histrange['prey1_scalar_projection_gradient']=np.arange(-200,200+100,100)

emdmats=dict()

allvars=['speed','speedgrad','accelmag','prey1reldistance','prey1reldistancegrad','prey1relangularhead','prey1relangularheadgrad','prey1_scalar_projection','prey1_scalar_projection_gradient']


#Compute recursive EMD for all above variables. It's fast afuck now
for varname in allvars: #Loop over variables
    dattmp=[] #list of dataframes
    for i in range(n_sessions): #loop over sessions and retrieve data
        session=i+1
        dat=getvarforclustering(datall[str(session)]['p1trialdat'],varname,winlength)
        dattmp.append([pd.DataFrame(dat)])

    #Now combine all sessions
    datcombo=pd.DataFrame()

    for i in range(n_sessions):
        datcombo=pd.concat((datcombo,dattmp[i][0]),axis=1)

    #Now compute CDFs as prep for wasserstein distance
    recdf = recusiveCDF(datcombo.to_numpy(),histrange,varname=varname)
    #Now compute earthmovers distance
    emdmat=EMDfast(recdf,normalize=False)
    emdmatrices[varname]=emdmat



In [ ]:
#Do affinity clustering
A=(emdmatrices['speed']*-1)/np.max(emdmatrices['speed'])
B=(emdmatrices['speedgrad']*-1)/np.max(emdmatrices['speedgrad'])
C=(emdmatrices['prey1_scalar_projection']*-1)/np.max(emdmatrices['prey1_scalar_projection'])
D=(emdmatrices['prey1_scalar_projection_gradient']*-1)/np.max(emdmatrices['prey1_scalar_projection_gradient'])
E=(emdmatrices['prey1relangularheadgrad']*-1)/np.max(emdmatrices['prey1relangularheadgrad'])

F=(emdmatrices['prey1reldistance']*-1)/np.max(emdmatrices['prey1reldistance'])


afmat=((A+B+C+D+E+F)/6)
#afmat=((A+B+C+D+E)/5)

af = AffinityPropagation(random_state=0,affinity='precomputed',damping=0.95).fit(afmat*-1)


In [ ]:
#Get data after clustering and group for mean, variance, and joint kernel density maps
dattmp=[] #list of dataframes
for i in range(n_sessions): #loop over sessions and retrieve data
    session=i+1
    dat=getvarforclustering(datall[str(session)]['p1trialdat'],varname,winlength)
    dattmp.append([pd.DataFrame(dat)])

#Now combine all sessions
datcombo=pd.DataFrame()

for i in range(n_sessions):
    datcombo=pd.concat((datcombo,dattmp[i][0]),axis=1)



#How to flatten out for plotting
datcombo.to_numpy()[:,np.where(af.labels_==3)[0]].transpose().flatten()

#cluster notes:
Accelmag and speedgradient are correlated a lot to the extent that you take the absolute value of speedgradient. May prefer speedgradient because it is directional (faster or slower).

In [ ]:
#Predefine bins for each variable type (either as a sequence or an integer)
#Does it matter about differing bins. This will kinda cause different resolutions....
# histrange=dict()
# histrange['speed']=np.arange(0,1400+100,200)
# histrange['speedgrad']=np.arange(-200,200+50,50)
# histrange['accelmag']=np.arange(0,20000+5000,5000)
# histrange['prey1reldistance']=np.arange(0,2000+200,200)
# histrange['prey1reldistancegrad']=np.arange(-40,40+10,20)
# histrange['prey1relangularhead']= np.arange(-180,180+45,45)
# histrange['prey1relangularheadgrad']= np.arange(-30,30+30,15)
# histrange['prey1_scalar_projection']=np.arange(-1500,1500+200,200)
# histrange['prey1_scalar_projection_gradient']=np.arange(-200,200+100,100)

#Fewer bins
histrange=dict()

histrange['speed']=np.arange(0,1400+300,400) # 4bins
histrange['speedgrad']=np.arange(-200,200+50,100) # 4bins
histrange['accelmag']=np.arange(0,20000+5000,5000) # 4bins
histrange['prey1reldistance']=np.arange(0,2000+400,400) # 5bins
histrange['prey1reldistancegrad']=np.arange(-40,40+10,20) #4 bins
histrange['prey1relangularhead']= np.arange(-180,180+45,45)
histrange['prey1relangularheadgrad']= np.arange(-30,30+30,15)
histrange['prey1_scalar_projection']=np.arange(-1500,1500+500,500)
histrange['prey1_scalar_projection_gradient']=np.arange(-200,200+100,100)


emdmatrices=dict()

allvars=['speed','speedgrad','accelmag','prey1reldistance','prey1reldistancegrad','prey1relangularhead','prey1relangularheadgrad','prey1_scalar_projection','prey1_scalar_projection_gradient']

#Compute recursive EMD for all above variables. It'll take a bit
for varname in allvars:
    #Make vectors of each variable and then reshape (turn into function)
    dat=getvarforclustering(p1trialdat,varname,winlength)
    recdf=recusiveCDF(dat,histrange,varname=varname)
    #Recursively compute earth movers distance. I could save time by recognizing it's symmetric and only compute unique entries (but I am lazy)..
    emdmat=EMDRecursive(recdf,normalize=False)
    emdmatrices[varname]=emdmat



#Save the matrices for later analysis
import pickle

with open('C:\\Users\\Justin\\Documents\\GitHub\\PacManHuman\\Results\\emdmats10.pkl', 'wb') as handle:
    pickle.dump(emdmatrices, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
#normalize range of earth-movers to be [-1,0]
emdmatrices=loadEMDmatrices('C:\\Users\\Justin\\Documents\\GitHub\\PacManHuman\\Results\\emdmats10.pkl')

varsinclude=['speed','speedgrad','prey1_scalar_projection','accelmag','prey1_scalar_projection_gradient','prey1reldistance','prey1relangularhead','prey1relangularheadgrad']

A=(emdmatrices['speed']*-1)/np.max(emdmatrices['speed'])
B=(emdmatrices['speedgrad']*-1)/np.max(emdmatrices['speedgrad'])
C=(emdmatrices['prey1_scalar_projection']*-1)/np.max(emdmatrices['prey1_scalar_projection'])
D=(emdmatrices['prey1_scalar_projection_gradient']*-1)/np.max(emdmatrices['prey1_scalar_projection_gradient'])
E=(emdmatrices['prey1relangularheadgrad']*-1)/np.max(emdmatrices['prey1relangularheadgrad'])

F=(emdmatrices['prey1reldistance']*-1)/np.max(emdmatrices['prey1reldistance'])


afmat=((A+B+C+D+E+F)/6)
afmat=((A+B+C+D+E)/5)

af = AffinityPropagation(random_state=0,affinity='precomputed',damping=0.95).fit(afmat*-1)

#mean variables now to make look at functional behavior based on clusters
behavecluster=dict() #Save these in a dictionary
for varsget in varsinclude:
    df=pd.DataFrame()
    for i in np.unique(af.labels_):
        dat=getvarforclustering(p1trialdat,varsget,winlength)
        tmp=pd.DataFrame((dat[:,np.where(af.labels_==i)]).reshape((winlength,len(np.where(af.labels_==i)[0]))).mean(axis=1))
        tmp.rename(columns={0:i},inplace=True)
        df=pd.concat((df,tmp),axis=1)
    behavecluster[varsget]=df


varsget='prey1reldistance'
dat=getvarforclustering(p1trialdat,varsget,winlength)
for i in range(5):
    sns.kdeplot(pd.DataFrame((dat[:, np.where(af.labels_ == i)]).reshape(
        (winlength, len(np.where(af.labels_ == i)[0])))).to_numpy().flatten())


#Build a 2d density plot with kdeplot
varsget='prey1reldistance'
dat=getvarforclustering(p1trialdat,varsget,winlength)

In [ ]:



import umap
import hdbscan
#UMAP plot with cluster labels
reducer = umap.UMAP(metric='precomputed',n_components=3,n_neighbors=25)
embedding=reducer.fit_transform(afmat*-1)




labels = hdbscan.HDBSCAN(
    min_samples=20,
    min_cluster_size=100,
).fit_predict(embedding)
clustered = (labels >= 0)
plt.scatter(embedding[clustered, 0],
            embedding[clustered, 1],
            c=labels[clustered],
            s=0.1,
            cmap='Spectral')
plt.show()


#mean variables now to make look at functional behavior based on clusters
behavecluster=dict() #Save these in a dictionary
for varsget in varsinclude:
    df=pd.DataFrame()
    for i in np.unique(labels):
        dat=getvarforclustering(p1trialdat,varsget,winlength)
        tmp=pd.DataFrame((dat[:,np.where(labels==i)]).reshape((winlength,len(np.where(labels==i)[0]))).mean(axis=1))
        tmp.rename(columns={0:i},inplace=True)
        df=pd.concat((df,tmp),axis=1)
    behavecluster[varsget]=df

#Compute transition probability, now (could do as a GLM to include condition informtion and neural data later).
#Get the probability distributions of variables for a given cluster

#Do certain clusters occur at certain trial points

In [ ]:
#Aggregate variables now to make look at functional behavior based on clusers


In [ ]:
#Compute transition probability, now (could do as a GLM to include condition informtion and neural data later)


In [ ]:
A=(emdmatrices['speed']*-1)/np.max(emdmatrices['speed'])
B=(emdmatrices['speedgrad']*-1)/np.max(emdmatrices['speedgrad'])
C=(emdmatrices['prey1_scalar_projection']*-1)/np.max(emdmatrices['prey1_scalar_projection'])
E=(emdmatrices['prey1_scalar_projection_gradient']*-1)/np.max(emdmatrices['prey1_scalar_projection_gradient'])
F=(emdmatrices['prey1reldistance']*-1)/np.max(emdmatrices['prey1reldistance'])

D=(emdmatrices['prey1reldistancegrad']*-1)/np.max(emdmatrices['prey1reldistancegrad'])

afmat=((A+B+C+D+E+F)/6)

#Let's try dim reduction/manifold learning first, then cluster
from sklearn.manifold import TSNE
import hdbscan

projection = TSNE(metric='precomputed',perplexity=30).fit_transform(afmat*-1)

labels = hdbscan.HDBSCAN(
    min_samples=20,
    min_cluster_size=30,
).fit_predict(projection)

clustered = (labels >= 0)

plt.scatter(projection[clustered, 0],
            projection[clustered, 1],
            c=labels[clustered],
            s=0.1,
            cmap='Spectral')
plt.show()


varsinclude=['speed','speedgrad','prey1_scalar_projection','prey1_scalar_projection_gradient','prey1reldistance','prey1reldistancegrad']


#mean variables now to make look at functional behavior based on clusters
behavecluster=dict() #Save these in a dictionary
for varsget in varsinclude:
    df=pd.DataFrame()
    for i in np.unique(labels):
        dat=getvarforclustering(p1trialdat,varsget,winlength)
        tmp=pd.DataFrame((dat[:,np.where(labels==i)]).reshape((winlength,len(np.where(labels==i)[0]))).mean(axis=1))
        tmp.rename(columns={0:i},inplace=True)
        df=pd.concat((df,tmp),axis=1)
    behavecluster[varsget]=df